<a href="https://colab.research.google.com/github/gregorywmorris/MLZoom2022/blob/main/Homework/MLZoomcampWeek3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Week 3 HW

Homework
Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

Dataset
In this homework, we will use the California Housing Prices data from Kaggle.

Here's a wget-able link:

wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
We'll keep working with the 'median_house_value' variable, and we'll transform it to a classification task.


# Features
For the rest of the homework, you'll need to use only these columns:

* 'latitude',
* 'longitude',
* 'housing_median_age',
* 'total_rooms',
* 'total_bedrooms',
* 'population',
* 'households',
* 'median_income',
* 'median_house_value',
* 'ocean_proximity'

In [5]:
import pandas as pd 
import numpy as np 

import seaborn as sns
from matplotlib import pyplot as pyplot
%matplotlib inline

In [6]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
!wget $data -O data.csv 

--2022-09-24 19:24:58--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.36M  --.-KB/s    in 0.05s   

2022-09-24 19:24:58 (28.1 MB/s) - ‘data.csv’ saved [1423529/1423529]



In [7]:
df = pd.read_csv('data.csv') 

# Data preparation
Select only the features from above and fill in the missing values with 0.
Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
Create a new column population_per_household by dividing the column population by the column households from dataframe.









In [8]:
select_features = ['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value',
'ocean_proximity']

In [9]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
  df[c] = df[c].str.lower().str.replace(' ', '_')

df.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [10]:
df = df[['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value',
'ocean_proximity']]

df.dtypes

latitude              float64
longitude             float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [11]:
df.fillna(0)

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,near_bay
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,near_bay
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,near_bay
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,near_bay
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,near_bay
...,...,...,...,...,...,...,...,...,...,...
20635,39.48,-121.09,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,inland
20636,39.49,-121.21,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,inland
20637,39.43,-121.22,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,inland
20638,39.43,-121.32,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,inland


In [12]:
df['bedrooms_per_room'] = df.total_bedrooms / df.total_rooms
df['population_per_household'] =  df.population / df.households
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,bedrooms_per_room,population_per_household
0,37.88,-122.23,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,near_bay,0.146591,2.555556
1,37.86,-122.22,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,near_bay,0.155797,2.109842
2,37.85,-122.24,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,near_bay,0.129516,2.802260
3,37.85,-122.25,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,near_bay,0.184458,2.547945
4,37.85,-122.25,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,near_bay,0.172096,2.181467


# Question 1
What is the most frequent observation (mode) for the column ocean_proximity?

<br>Options:

* NEAR BAY
* **< 1H OCEAN**
* INLAND
* NEAR OCEAN

In [13]:
df.ocean_proximity.mode()

0    <1h_ocean
dtype: object

# Split the data
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value (median_house_value) is not in your dataframe.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [15]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_full_train = df_full_train.reset_index(drop=True)

y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [16]:
len(df) == len(df_train)+len(df_val)+len(df_test)

True

# Question 2
Create the correlation matrix for the numerical features of your train dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
What are the two features that have the biggest correlation in this dataset?

<br>Options:

* **total_bedrooms and households**
* total_bedrooms and total_rooms
* population and households
* population_per_household and total_rooms

In [17]:
corr_matrix = df.corr().abs() # absolute value, we want to see the biggest correlation, direction doesn't matter for this question.

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

corr_values = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) #<- without .astype(bool) you get an error'Boolean array expected for the condition, not float64'
                  .stack() # Formats the data as you see below
                  .sort_values(ascending=False) # sorts highest to lowest.
                  .head()) 

#first element of sol series is the pair with the biggest correlation
corr_values


total_bedrooms  households        0.979728
total_rooms     total_bedrooms    0.930380
latitude        longitude         0.924664
total_rooms     households        0.918484
population      households        0.907222
dtype: float64

In [18]:
corr_matrix # this is why we use the .triu, notice if we get everying from the upper triangle we don't get duplicates. 

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,bedrooms_per_room,population_per_household
latitude,1.000000,0.924664,0.011173,0.036100,0.066983,0.108785,0.071035,0.079809,0.144160,0.113815,0.002366
longitude,0.924664,1.000000,0.108197,0.044568,0.069608,0.099773,0.055310,0.015176,0.045967,0.092657,0.002476
housing_median_age,0.011173,0.108197,1.000000,0.361262,0.320451,0.296244,0.302916,0.119034,0.105623,0.136089,0.013191
total_rooms,0.036100,0.044568,0.361262,1.000000,0.930380,0.857126,0.918484,0.198050,0.134153,0.187900,0.024581
total_bedrooms,0.066983,0.069608,0.320451,0.930380,1.000000,0.877747,0.979728,0.007723,0.049686,0.084238,0.028355
population,0.108785,0.099773,0.296244,0.857126,0.877747,1.000000,0.907222,0.004834,0.024650,0.035319,0.069863
households,0.071035,0.055310,0.302916,0.918484,0.979728,0.907222,1.000000,0.013033,0.065843,0.065087,0.027309
median_income,0.079809,0.015176,0.119034,0.198050,0.007723,0.004834,0.013033,1.000000,0.688075,0.615661,0.018766
median_house_value,0.144160,0.045967,0.105623,0.134153,0.049686,0.024650,0.065843,0.688075,1.000000,0.255880,0.023737
bedrooms_per_room,0.113815,0.092657,0.136089,0.187900,0.084238,0.035319,0.065087,0.615661,0.255880,1.000000,0.002938


# Make median_house_value binary
* We need to turn the median_house_value variable from numeric into binary.

* Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.


In [19]:
df_full_train['above_average'] = (df_full_train.median_house_value > df_full_train.median_house_value.mean())
df_full_train['above_average'] = (df_full_train['above_average']==True).astype(int)
df_full_train['above_average']

0        0
1        1
2        0
3        0
4        0
        ..
16507    1
16508    0
16509    1
16510    1
16511    1
Name: above_average, Length: 16512, dtype: int64

In [ ]:
above_average = (y_train > y_train.mean())

In [ ]:
above_average = (above_average==True).astype(int)

# Question 3
Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
What is the value of mutual information?
Round it to 2 decimal digits using round(score, 2)

<br>Options:

* 0.26
* 0
* **0.10**
* 0.16

In [20]:
from sklearn.metrics import mutual_info_score 

In [21]:
def mutual_info_median_score(series):
  return mutual_info_score(series,df_full_train.above_average)

In [22]:
round(df_full_train.apply(mutual_info_median_score).where(df_full_train.columns == 'ocean_proximity'),2)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

latitude                    NaN
longitude                   NaN
housing_median_age          NaN
total_rooms                 NaN
total_bedrooms              NaN
population                  NaN
households                  NaN
median_income               NaN
median_house_value          NaN
ocean_proximity             0.1
bedrooms_per_room           NaN
population_per_household    NaN
above_average               NaN
dtype: float64

# Question 4
* Now let's train a logistic regression
* Remember that we have one categorical variable ocean_proximity in the data; include it using one-hot encoding.
* Fit the model on the **training dataset**.
* To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the **validation dataset** and round it to 2 decimal digits.

<br>Options:

* 0.60
* 0.72
* 0.84
* 0.95

In [23]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [24]:
dv = DictVectorizer(sparse = False)

In [25]:
train_dict = df_train.to_dict(orient = 'records')

In [26]:
dv.fit(train_dict)
x_train = dv.transform(train_dict)

In [27]:
x_train.shape

(12384, 15)

In [28]:
x_train = dv.fit_transform(train_dict) # <- shorter method

In [29]:
x_train.shape

(12384, 15)

In [30]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)

In [ ]:
model.fit(x_train, y_train)

# Question 5
* Let's find the least useful feature using the feature elimination technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

<br> Which of following feature has the smallest difference?

* total_rooms
* total_bedrooms
* population
* households
* **note: the difference doesn't have to be positive

# Question 6
* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
* This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
* If there are multiple options, select the smallest alpha.

<br> Options:

* 0
* 0.01
* 0.1
* 1
* 10

# Submit the results
Submit your results here: https://forms.gle/vQXAnQDeqA81HSu86
You can submit your solution multiple times. In this case, only the last submission will be used
<br>If your answer doesn't match options exactly, select the closest one
<br>
#Deadline
The deadline for submitting is 26 September (Monday), 23:00 CEST.

After that, the form will be closed